import the data

In [62]:
import pandas as pd

In [63]:
df = pd.read_csv("result.csv", delimiter=",")
df

,episode,step,reward,info,timestamp,obs_0,obs_1,obs_2,obs_3,obs_4,...,obs_309,obs_310,obs_311,obs_312,obs_313,obs_314,obs_315,obs_316,obs_317,obs_318
0,0.0,0.0,-1.389655,0.0,1.679650e+09,0.013682,0.013316,-0.021371,-0.029317,0.001221,...,0.009389,-0.535267,-0.001717,0.844681,0.844631,-0.012196,0.535210,0.009383,0.999924,0.007979
1,0.0,1.0,-1.513281,0.0,1.679650e+09,0.043853,0.042083,-0.056912,-0.077301,0.003332,...,0.025483,-0.528323,-0.002555,0.849039,0.848661,-0.031585,0.527993,0.025468,0.999498,0.018856
2,0.0,2.0,-1.559966,0.0,1.679650e+09,0.084186,0.080007,-0.089216,-0.121493,-0.008056,...,0.031378,-0.530603,-0.002908,0.847615,0.847040,-0.038816,0.530110,0.031360,0.999242,0.023059
3,0.0,3.0,-1.557379,0.0,1.679650e+09,0.130438,0.123797,-0.110976,-0.150377,-0.028199,...,0.021509,-0.546160,-0.000571,0.837681,0.837405,-0.026035,0.545963,0.021497,0.999661,0.014698
4,0.0,4.0,-1.514907,0.0,1.679650e+09,0.179715,0.170944,-0.118389,-0.157974,-0.052214,...,-0.005936,-0.575679,0.005210,0.817660,0.817655,0.010918,0.575606,-0.005928,0.999927,-0.010545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9.0,195.0,1.643387,0.0,1.679650e+09,0.016023,-0.021945,-0.223133,-0.128909,-0.091112,...,0.092495,-0.322932,-0.032672,0.945858,0.941897,-0.108712,0.317824,0.092442,0.993536,0.065880
1996,9.0,196.0,1.642209,0.0,1.679650e+09,0.011088,-0.025304,-0.222518,-0.133515,-0.092572,...,0.099743,-0.319556,-0.033837,0.946963,0.942309,-0.116484,0.313823,0.099688,0.992616,0.069108
1997,9.0,197.0,1.642716,0.0,1.679650e+09,0.007984,-0.027461,-0.222570,-0.145634,-0.094361,...,0.112104,-0.318070,-0.036716,0.947356,0.941423,-0.130324,0.311027,0.112044,0.990791,0.076017
1998,9.0,198.0,1.634751,0.0,1.679650e+09,0.004195,-0.029033,-0.222018,-0.171272,-0.096102,...,0.133622,-0.317409,-0.042347,0.947343,0.938837,-0.154726,0.307643,0.133551,0.987049,0.088868


build the network

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

In [77]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
from sklearn.metrics import mean_squared_error

In [65]:
X = pd.concat([df.iloc[:,4:46],df.iloc[:,46:]],axis=1,ignore_index=True)

y = df.iloc[:,45]

In [66]:
X = torch.tensor(X.values.astype(np.float32))
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [67]:
X.shape

torch.Size([2000, 320])

In [68]:
y.shape

torch.Size([2000, 1])

In [69]:
model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid())

In [70]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(320, 160)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(160, 80)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(80, 16)
        self.act_output =  nn.Linear(16, 1)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=320, out_features=160, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=160, out_features=80, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=80, out_features=16, bias=True)
  (act_output): Linear(in_features=16, out_features=1, bias=True)
)


In [71]:
y[0:10]

tensor([[-1.5550e-18],
        [-2.4689e-17],
        [-9.8585e-17],
        [-2.3664e-16],
        [-1.2744e-06],
        [-9.1876e-06],
        [-1.4657e-05],
        [-1.7064e-05],
        [ 3.7224e-03],
        [ 1.5565e-02]])

In [72]:
loss_fn = nn.MSELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 7748.1298828125
Finished epoch 1, latest loss 0.41201943159103394
Finished epoch 2, latest loss 0.01066088117659092
Finished epoch 3, latest loss 1.9202991724014282
Finished epoch 4, latest loss 0.8531777262687683
Finished epoch 5, latest loss 0.6139273643493652
Finished epoch 6, latest loss 0.08486442267894745
Finished epoch 7, latest loss 0.48295658826828003
Finished epoch 8, latest loss 0.3270663321018219
Finished epoch 9, latest loss 0.34234756231307983
Finished epoch 10, latest loss 162.12570190429688
Finished epoch 11, latest loss 1344.9486083984375
Finished epoch 12, latest loss 39550.734375
Finished epoch 13, latest loss 8.149935722351074
Finished epoch 14, latest loss 0.4979695677757263
Finished epoch 15, latest loss 0.22413849830627441
Finished epoch 16, latest loss 16084699.0
Finished epoch 17, latest loss 47.94384765625
Finished epoch 18, latest loss 0.2372269332408905
Finished epoch 19, latest loss 176.19418334960938
Finished epoch 20, latest 

In [78]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
 
trainScore = math.sqrt(mean_squared_error(y_pred,y))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 65.57 RMSE
